In [7]:
import pandas as pd
import numpy as np
import urllib.request
import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB 
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
from pymystem3 import Mystem
from stop_words import get_stop_words
from transformers import pipeline
import json
import os
from os.path import join

In [34]:
file_path = join(os.environ["HOME"], "Datasets/ant.json")

with open(file_path, "r") as json_file:
    ant = json.load(json_file)

file_path = join(os.environ["HOME"], "Datasets/ant_adv.json")

with open(file_path, "r") as json_file:
    ant_adv = json.load(json_file)

ant_adv

{'дешево': 'некалиброванной',
 'интересно': 'интересно',
 'вкусно': 'свежий',
 'прано': 'прано',
 'ппомимо': 'ппомимо',
 'гармонично': 'гармонично',
 'сильно': 'сильно',
 'сюда': 'отсутствие',
 'однозначно': 'однозначно',
 'домой': 'середина',
 'приятно': 'приятно',
 'отлично': 'проницаемой',
 'вовремя': 'вовремя',
 'быстро': 'постепенный',
 'вновь': 'возраст',
 'отдельно': 'отдельно',
 'обязательно': 'обязательно',
 'заранее': 'пребывание',
 'хорошо': 'выживать',
 'шумно': 'шумно',
 'важно': 'несущественно',
 'легко': 'сложность',
 'удачно': 'удачно',
 'неожиданно': 'взимается',
 'впервые': 'опоздавший',
 'классно': 'демилитаризацию',
 'наедине': 'распределены',
 'побязательно': 'побязательно',
 'еще': 'еще',
 'прямо': 'несоблюдение',
 'д': 'д',
 'вполне': 'неумеренно',
 'весьма': 'неискренний',
 'чисто': 'чисто',
 'пчто': 'пчто',
 'удобно': 'удобно',
 'буквально': 'буквально',
 'сразу': 'свидетельствует',
 'поднозначно': 'поднозначно',
 'слегка': 'неумеренно',
 'зря': 'зря',
 'высоко

In [88]:
len(ant_adv)

1443

In [10]:
data_test_filename = join(os.environ["HOME"], "Datasets/yandex_data_for_llm.csv")
df_test = pd.read_csv(data_test_filename, sep=',', index_col=False, usecols=lambda column: column not in ['Unnamed: 0'])
df_test

,rating,text
0,5.0,Приятное место. Обедали компанией в субботу. М...
1,4.0,"Хорошее кафе, есть для детей игровой уголок и ..."
2,4.0,\n4 с минусом. Туалет это дичь просто. Если ты...
3,5.0,Прекрасное заведение и атмосфера. Все было оче...
4,5.0,"Вежливый персонал, быстрая скорость отдачи зак..."
...,...,...
27655,5.0,"Заказала доставку, всё привезли хорошо упакова..."
27656,5.0,Доброжелательное место с отличным интерьером и...
27657,5.0,Хорший ресторанчик на набережнрй Города-Героя ...
27658,5.0,"Вкусное кафе, относительно демократичные цены,..."


In [11]:
data_train_filename = join(os.environ["HOME"], "Datasets/yandex_data_train.csv")
df_train = pd.read_csv(data_train_filename, sep=',', index_col=False, usecols=lambda column: column not in ['Unnamed: 0'])
df_train

,rating,text
0,4.0,"Ресторан в центре Москвы, в замечательном мест..."
1,4.0,"Кухня не плохая, довольно таки вкусно. Но для ..."
2,5.0,"Отличный ресторан, попробовали очень много блю..."
3,4.0,"Бизнес-встречу здесь планировать провально , а..."
4,5.0,Завтракали в этом ресторане при гостинице. Все...
...,...,...
64532,4.0,"Чтобы получить отзыв 5,0 нужно очень постарат..."
64533,5.0,Очень уютное кафе! Была там несколько раз. Пер...
64534,3.0,"Невозможно заказать с доставкой, мало мест для..."
64535,5.0,"Доброжелательный персонал. Вкусная еда, хороша..."


In [12]:
stop_words = get_stop_words('russian')
def remove_russian_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

mystem = Mystem()
# Function to apply stemming to Russian text
def stem_russian_text(text):
    tokens = mystem.lemmatize(text)
    return ''.join(tokens)


def prepare(data):
    # Step 1: Remove punctuation from the text column
    data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

    # Step 2: Remove stop words from the text column
    data['text'] = data['text'].apply(remove_russian_stop_words)
    data['text'] = data['text'].apply(stem_russian_text)


In [13]:
prepare(df_test)
prepare(df_train)

In [35]:
from nltk.tokenize import word_tokenize

# Define a function to swap adjectives with their antonyms in a text
def swap_adjectives(text):
    tokens = word_tokenize(text, language='russian')
    swapped_text = []
    for token in tokens:
        if token in ant:
            swapped_text.append(ant[token])
        elif token in ant_adv:
            swapped_text.append(ant_adv[token])
        else:
            swapped_text.append(token)
    return ' '.join(swapped_text)

# Example usage
text = "Большой красивый дом стоит на холме."
swapped_text = swap_adjectives(text)
print(swapped_text)

Большой непривлекательным дом стоит на холме .


In [36]:
df_test["text"]

0        приятный место обедать компания суббота место ...
1        хороший кафе ребенок игровой уголок детский ме...
2        n4 минус туалет дичь мужчина вставать унитаз п...
3        прекрасный заведение атмосфера вкусно божестве...
4        вежливый персонал быстрый скорость отдача зака...
                               ...                        
27655    заказывать доставка привозить упаковывать быст...
27656    доброжелательный место отличный интерьер музык...
27657    хорший ресторанчик набережнрй город герой прек...
27658    вкусный кафе относительно демократичный цена п...
27659    отличный кафе кухня супер вкусно быстро владик...
Name: text, Length: 27660, dtype: object

In [37]:
df_test["text_ant"] = df_test['text'].apply(swap_adjectives)

In [38]:
df_all = pd.concat([df_train['text'], df_test['text'], df_test['text_ant']])
df_all

0        ресторан центр москва замечательный место хоро...
1        кухня плохой таки вкусно обстановка подвал дор...
2        отличный ресторан попробовать блюдо вкусно пер...
3        бизнес встреча планировать провальный проводит...
4        завтракать ресторан гостиница невероятно вкусн...
                               ...                        
27655    заказывать доставка привозить упаковывать пост...
27656    необщительный место маленький интерьер музыка ...
27657    хорший ресторанчик набережнрй город герой непр...
27658    свежий кафе относительно недемократично цена п...
27659    маленький кафе кухня низкий свежий постепенный...
Length: 119857, dtype: object

In [63]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_all)

TfidfVectorizer()

In [49]:
X_train = vectorizer.transform(df_train['text'])
y_train = df_train['rating'].apply(lambda x: x > 4)

X_test = vectorizer.transform(df_test['text'])
X_drift_tesst = vectorizer.transform(df_test['text_ant'])
y_test = df_test['rating'].apply(lambda x: x > 4)

In [50]:
clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.2)

In [51]:
# NO DRIFT
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8841287057122198
              precision    recall  f1-score   support

  Not 5 star       0.92      0.57      0.71      6720
      5 star       0.88      0.98      0.93     20940

    accuracy                           0.88     27660
   macro avg       0.90      0.78      0.82     27660
weighted avg       0.89      0.88      0.87     27660



In [56]:
#  DRIFT
y_pred = clf.predict(X_drift_tesst)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.9053145336225596
              precision    recall  f1-score   support

  Not 5 star       0.94      0.65      0.77      6720
      5 star       0.90      0.99      0.94     20940

    accuracy                           0.91     27660
   macro avg       0.92      0.82      0.86     27660
weighted avg       0.91      0.91      0.90     27660



In [53]:
X_test_retrain, X_test_30, y_test_retrain, y_test_30 = train_test_split(X_drift_tesst, y_test, test_size=0.3, random_state=42)

In [54]:
clf.partial_fit(X_test_retrain, y_test_retrain)

MultinomialNB(alpha=0.2)

In [55]:
y_pred = clf.predict(X_test_30)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test_30, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_30, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8838274282959749
              precision    recall  f1-score   support

  Not 5 star       0.89      0.59      0.71      2000
      5 star       0.88      0.98      0.93      6298

    accuracy                           0.88      8298
   macro avg       0.89      0.78      0.82      8298
weighted avg       0.88      0.88      0.87      8298



In [78]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean

class HDDMText:
    def __init__(self, vec, threshold=0.1):
        self.threshold = threshold
        self.reference_distribution = None
        self.vectorizer = vec

    def _compute_hellinger_distance(self, p, q):
        return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q))**2)) / np.sqrt(2)

    def _preprocess_text(self, text_data):
        return self.vectorizer.transform(text_data)

    def detect_drift(self, period1_text, period2_text):
        if self.reference_distribution is None:
            self.reference_distribution = self._preprocess_text(period1_text)
            return False

        p = self._preprocess_text(period1_text)
        q = self._preprocess_text(period2_text)

        # Convert to dense array for Hellinger distance calculation
        p_dense = p.toarray().mean(axis=0)
        q_dense = q.toarray().mean(axis=0)

        hellinger_distance = self._compute_hellinger_distance(p_dense, q_dense)

        print(323)
        return hellinger_distance > self.threshold

period1_text = list(df_train['text'])
period2_text = list(df_test['text_ant'])

# Initialize HDDMText
hddm_text = HDDMText(vectorizer, threshold=0.1)

# Detect drift
is_drift = hddm_text.detect_drift(period1_text, period2_text)

if is_drift:
    print("Drift detected!")
else:
    print("No drift detected.")


In [81]:
period1_text = list(df_train['text'])
period2_text = list(df_test['text_ant'])

# Initialize HDDMText
hddm_text2 = HDDMText2(vectorizer, threshold=0.1)

# Detect drift
is_drift = hddm_text2.detect_drift(period1_text, period2_text)

print(is_drift)

False


In [86]:
period1_text = df_train['text']
period2_text = df_test['text_ant']

p = vectorizer.transform(period1_text)
q = vectorizer.transform(period2_text)

# Convert to dense array for Hellinger distance calculation
p = p.toarray().mean(axis=0)
q = q.toarray().mean(axis=0)
np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q))**2)) / np.sqrt(2)

1.0083788511523823

In [85]:
import pandas as pd
from collections import Counter
import numpy as np
from scipy.stats import ks_2samp

def calculate_word_frequencies(text_data):
    # Tokenize text and count word frequencies
    tokenized_text = [word for sentence in text_data for word in sentence.split()]
    word_freq = Counter(tokenized_text)
    return word_freq

def detect_data_drift(part1_data, part2_data):
    # Calculate word frequencies for each part
    part1_word_freq = calculate_word_frequencies(part1_data)
    part2_word_freq = calculate_word_frequencies(part2_data)

    # Extract word frequencies and sort by word
    part1_word_freq_values = np.array(list(part1_word_freq.values()))
    part2_word_freq_values = np.array(list(part2_word_freq.values()))

    # Calculate Kolmogorov-Smirnov statistic
    ks_statistic, p_value = ks_2samp(part1_word_freq_values, part2_word_freq_values)

    # Define a threshold for significant drift (you can adjust this threshold)
    threshold = 0.05
    return p_value

    # Check for data drift
    if p_value < threshold:
        print("Data drift detected!")
    else:
        print("No significant data drift detected.")

# Sample data (replace with your dataset)
part1_data = list(df_train['text'])
part2_data = list(df_test['text_ant'])

# Detect data drift
detect_data_drift(part1_data, part2_data)

0.20913098383007178